# BIOL550 Project Dataset Analysis Notebook

**Selected projects:** `PRJNA717662`, `PRJNA1017789`, `PRJNA1277581`

## Goal
Build a reproducible workflow to collect SRA metadata, validate each project against BIOL550 dataset requirements, and prepare the datasets for downstream RNA-seq analysis.


## Section 1 - Project Datasets Selected


### 1. PRJNA717662 - Long-term Blood Transcriptome after SARS-CoV-2
- **Project Accession:** `PRJNA717662`
- **BioProject:** https://www.ncbi.nlm.nih.gov/bioproject/PRJNA717662
- **SRA Study (SRP312283):** https://trace.ncbi.nlm.nih.gov/Traces/study/?acc=SRP312283
- **Description:** Profiles blood transcriptomes from recovered COVID-19 participants at multiple post-infection timepoints to identify persistent immune/transcriptional signatures.
- **Sample Size:** 152 SRA runs (`Homo sapiens`, paired-end).
- **Read Depth:** All 152 runs are >=40M spots (median: ~63.6M; range: ~43.8M to ~108.3M). Average spot length is ~300 bp (mostly 2x150).
- **Case/Control:** Includes healthy controls and post-infection samples at 12, 16, and 24 weeks.
- **Strengths:**
  - Large cohort and deep sequencing.
  - Public access and consistent Illumina NovaSeq 6000 platform.
  - Strong timepoint-based study design.
- **Weaknesses:**
  - Blood RNA can include strong immune-cell composition effects.
  - Downstream contrasts require careful timepoint/clinical grouping.


### 2. PRJNA1017789 - AhR and Axon Regeneration in DRG Neurons
- **Project Accession:** `PRJNA1017789`
- **BioProject:** https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1017789
- **SRA Study (SRP460727):** https://trace.ncbi.nlm.nih.gov/Traces/study/?acc=SRP460727
- **Description:** Mouse DRG RNA-seq study testing how aryl hydrocarbon receptor (AhR) affects stress signaling and axon regeneration after injury.
- **Sample Size:** 26 SRA runs (`Mus musculus`, paired-end).
- **Read Depth:** All 26 runs are >=40M spots (median: ~42.8M; range: ~40.1M to ~65.4M). Average spot length is 302 bp (~2x151).
- **Case/Control:** Contains genotype and injury-based groups (Ahr cKO vs controls; naive vs post-axotomy).
- **Strengths:**
  - Meets depth/layout requirements with consistent transcriptomic libraries.
  - Clear perturbation design for differential expression.
  - Publicly accessible runs.
- **Weaknesses:**
  - Smaller cohort than PRJNA717662.
  - More complex multi-factor contrasts (genotype + injury + batch/zeitgeber timing).


### 3. PRJNA1277581 - Muller Glia-Microglia Crosstalk in Retina Regeneration
- **Project Accession:** `PRJNA1277581`
- **BioProject:** https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1277581
- **SRA Study (SRP592470):** https://trace.ncbi.nlm.nih.gov/Traces/study/?acc=SRP592470
- **Description:** Zebrafish retina regeneration RNA-seq study assessing microglia effects on Muller glia reprogramming and cell-division pathways.
- **Sample Size:** 30 SRA runs (`Danio rerio`, paired-end).
- **Read Depth:** All 30 runs are >=40M spots (median: ~62.4M; range: ~50.9M to ~173.3M). Average spot length is 302 bp (~2x151).
- **Case/Control:** Includes uninjured controls, injury models, and microglia-perturbation conditions across two experimental batches.
- **Strengths:**
  - High-depth sequencing with strong perturbation design.
  - Good sample count and clearly defined treatment groups.
  - Suitable for regeneration-focused pathway analyses.
- **Weaknesses:**
  - Cross-batch harmonization is important (RS14 vs RS19 design).
  - Non-human organism requires zebrafish-specific annotation references.


## Section 2 - Data Collection (RunInfo Pull + Summary)
This section uses the same Entrez Direct workflow from project preparation:

- `esearch -db sra -query "<BioProject>" | efetch -format runinfo > <BioProject>_runinfo.csv`


In [9]:
from pathlib import Path
import csv
import statistics
from collections import Counter
import pandas as pd

PROJECTS = ["PRJNA717662", "PRJNA1017789", "PRJNA1277581"]
SRP_BY_PROJECT = {
    "PRJNA717662": "SRP312283",
    "PRJNA1017789": "SRP460727",
    "PRJNA1277581": "SRP592470",
}

LAB_DIR = Path('/Users/pitergarcia/DataScience/Semester5/BIOL550/BIOL550-Lab')
DATA_DIR = LAB_DIR / 'project_pic'
DATA_DIR.mkdir(parents=True, exist_ok=True)


In [10]:
%%bash
set -euo pipefail

LOCAL_DIR="/Users/pitergarcia/DataScience/Semester5/BIOL550/BIOL550-Lab/project_pic"
REL_DIR_1="Semester5/BIOL550/BIOL550-Lab/project_pic"
REL_DIR_2="project_pic"

if [ -d "$LOCAL_DIR" ]; then
  cd "$LOCAL_DIR"
elif [ -d "$REL_DIR_1" ]; then
  cd "$REL_DIR_1"
elif [ -d "$REL_DIR_2" ]; then
  cd "$REL_DIR_2"
else
  echo "Could not locate project_pic directory."
  echo "Checked: $LOCAL_DIR, $REL_DIR_1, $REL_DIR_2"
  exit 1
fi

have_entrez=false
if command -v esearch >/dev/null 2>&1 && command -v efetch >/dev/null 2>&1; then
  have_entrez=true
fi

have_curl=false
if command -v curl >/dev/null 2>&1; then
  have_curl=true
fi

have_wget=false
if command -v wget >/dev/null 2>&1; then
  have_wget=true
fi

if [ "$have_entrez" = false ] && [ "$have_curl" = false ] && [ "$have_wget" = false ]; then
  echo "No supported downloader found. Install one of:"
  echo "- Entrez Direct (esearch + efetch)"
  echo "- curl"
  echo "- wget"
  exit 1
fi

for acc in PRJNA717662 PRJNA1017789 PRJNA1277581; do
  echo "Downloading RunInfo for $acc"

  if [ "$have_entrez" = true ]; then
    esearch -db sra -query "$acc" | efetch -format runinfo > "${acc}_runinfo.csv"
  elif [ "$have_curl" = true ]; then
    curl -fsSL "https://trace.ncbi.nlm.nih.gov/Traces/sra-db-be/runinfo?acc=${acc}" -o "${acc}_runinfo.csv"
  else
    wget -qO "${acc}_runinfo.csv" "https://trace.ncbi.nlm.nih.gov/Traces/sra-db-be/runinfo?acc=${acc}"
  fi

  if [ ! -s "${acc}_runinfo.csv" ]; then
    echo "Failed: empty file for $acc"
    exit 1
  fi

done


In [12]:
def to_num(v):
    try:
        if v is None or v == "":
            return None
        return float(v)
    except Exception:
        return None


def summarize_runinfo(csv_path):
    with open(csv_path, newline="") as f:
        rows = list(csv.DictReader(f))

    spots = [to_num(r.get("spots")) for r in rows]
    spots = [x for x in spots if x is not None]
    avg_len = [to_num(r.get("avgLength")) for r in rows]
    avg_len = [x for x in avg_len if x is not None]

    layout = Counter((r.get("LibraryLayout") or "") for r in rows)
    org = Counter((r.get("ScientificName") or "") for r in rows)

    return {
        "runs": len(rows),
        "paired_runs": layout.get("PAIRED", 0),
        "top_organism": org.most_common(1)[0][0] if org else "",
        "median_spots": int(statistics.median(spots)) if spots else None,
        "min_spots": int(min(spots)) if spots else None,
        "max_spots": int(max(spots)) if spots else None,
        "median_avg_len": round(statistics.median(avg_len), 1) if avg_len else None,
    }


print("Project	Runs	Paired	Top organism	Median spots	Min spots	Max spots	Median avgLength")
for acc in PROJECTS:
    s = summarize_runinfo(DATA_DIR / f"{acc}_runinfo.csv")
    print(
        f"{acc}	{s['runs']}	{s['paired_runs']}	{s['top_organism']}	"
        f"{s['median_spots']}	{s['min_spots']}	{s['max_spots']}	{s['median_avg_len']}"
    )


Project	Runs	Paired	Top organism	Median spots	Min spots	Max spots	Median avgLength
PRJNA717662	152	152	Homo sapiens	63609267	43836864	108271493	300.0
PRJNA1017789	26	26	Mus musculus	42807360	40136901	65409634	302.0
PRJNA1277581	30	30	Danio rerio	62441451	50911670	173268353	302.0


### Data Collection Summary (RunInfo Tables)
This section stores RunInfo metadata in pandas objects and reports a general summary for each selected project.


In [14]:
from IPython.display import display

# Project-level table (one row per selected project)
project_manifest = pd.DataFrame([
    {
        "BioProject": acc,
        "SRAStudy": SRP_BY_PROJECT[acc],
        "BioProjectURL": f"https://www.ncbi.nlm.nih.gov/bioproject/{acc}",
        "SRAStudyURL": f"https://trace.ncbi.nlm.nih.gov/Traces/study/?acc={SRP_BY_PROJECT[acc]}",
        "RunInfoCSV": str(DATA_DIR / f"{acc}_runinfo.csv"),
    }
    for acc in PROJECTS
]).sort_values(["BioProject"]).reset_index(drop=True)

# Run-level table (one row per SRR run across all selected projects)
run_tables = []
for acc in PROJECTS:
    df = pd.read_csv(DATA_DIR / f"{acc}_runinfo.csv")
    df["BioProject"] = acc
    run_tables.append(df)

run_manifest = pd.concat(run_tables, ignore_index=True)
run_manifest = run_manifest[[
    "BioProject", "SRAStudy", "Run", "ScientificName", "LibraryLayout",
    "spots", "avgLength", "Platform", "Model", "BioSample"
]].copy()

summary = (
    run_manifest
    .groupby(["BioProject", "SRAStudy"], as_index=False)
    .agg(
        runs=("Run", "count"),
        paired_runs=("LibraryLayout", lambda s: (s == "PAIRED").sum()),
        median_spots=("spots", "median"),
        min_spots=("spots", "min"),
        max_spots=("spots", "max"),
        median_avgLength=("avgLength", "median"),
    )
    .sort_values(["BioProject"])
    .reset_index(drop=True)
)

summary["pairing_rate"] = (summary["paired_runs"] / summary["runs"] * 100).round(1)

project_manifest_view = project_manifest.rename(columns={
    "BioProject": "BioProject",
    "SRAStudy": "SRA Study",
    "BioProjectURL": "BioProject URL",
    "SRAStudyURL": "SRA Study URL",
    "RunInfoCSV": "RunInfo CSV",
})

summary_view = summary.rename(columns={
    "runs": "Runs",
    "paired_runs": "Paired Runs",
    "pairing_rate": "Pairing %",
    "median_spots": "Median Spots",
    "min_spots": "Min Spots",
    "max_spots": "Max Spots",
    "median_avgLength": "Median avgLength",
})

for col in ["Median Spots", "Min Spots", "Max Spots"]:
    summary_view[col] = summary_view[col].round(0).astype(int).map(lambda x: f"{x:,}")
summary_view["Median avgLength"] = summary_view["Median avgLength"].round(1)
summary_view["Pairing %"] = summary_view["Pairing %"].map(lambda x: f"{x:.1f}%")

print("Project Manifest")
display(project_manifest_view)

print("RunInfo Summary")
display(summary_view)


Project Manifest


,BioProject,SRA Study,BioProject URL,SRA Study URL,RunInfo CSV
0,PRJNA1017789,SRP460727,https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1...,https://trace.ncbi.nlm.nih.gov/Traces/study/?a...,/Users/pitergarcia/DataScience/Semester5/BIOL5...
1,PRJNA1277581,SRP592470,https://www.ncbi.nlm.nih.gov/bioproject/PRJNA1...,https://trace.ncbi.nlm.nih.gov/Traces/study/?a...,/Users/pitergarcia/DataScience/Semester5/BIOL5...
2,PRJNA717662,SRP312283,https://www.ncbi.nlm.nih.gov/bioproject/PRJNA7...,https://trace.ncbi.nlm.nih.gov/Traces/study/?a...,/Users/pitergarcia/DataScience/Semester5/BIOL5...


RunInfo Summary


,BioProject,SRAStudy,Runs,Paired Runs,Median Spots,Min Spots,Max Spots,Median avgLength,Pairing %
0,PRJNA1017789,SRP460727,26,26,"42,807,360","40,136,901","65,409,634",302.0,100.0%
1,PRJNA1277581,SRP592470,30,30,"62,441,452","50,911,670","173,268,353",302.0,100.0%
2,PRJNA717662,SRP312283,152,152,"63,609,267","43,836,864","108,271,493",300.0,100.0%


## Section 3 - Data Exploration and Requirement Validation
Validation checks based on `task_n_desc.md` project-selection constraints:

- At least 20 runs/samples.
- Paired-end libraries.
- Average read length near 2x150 (use threshold `avgLength >= 300`).
- Sufficient depth (`spots >= 40,000,000`).


In [20]:
from IPython.display import display

MIN_RUNS = 20
MIN_SPOTS = 40_000_000
MIN_AVG_LENGTH = 300


def validate_project(csv_path):
    with open(csv_path, newline="") as f:
        rows = list(csv.DictReader(f))

    total_runs = len(rows)
    paired_runs = sum(1 for r in rows if (r.get("LibraryLayout") or "") == "PAIRED")

    spots_vals = [to_num(r.get("spots")) for r in rows]
    avg_vals = [to_num(r.get("avgLength")) for r in rows]

    spots_meet = [r.get("Run", "") for r, s in zip(rows, spots_vals) if s is not None and s >= MIN_SPOTS]
    spots_under = [r.get("Run", "") for r, s in zip(rows, spots_vals) if s is None or s < MIN_SPOTS]

    avg_meet = [r.get("Run", "") for r, a in zip(rows, avg_vals) if a is not None and a >= MIN_AVG_LENGTH]
    avg_under = [r.get("Run", "") for r, a in zip(rows, avg_vals) if a is None or a < MIN_AVG_LENGTH]

    return {
        "total_runs": total_runs,
        "paired_runs": paired_runs,
        "passes_min_runs": total_runs >= MIN_RUNS,
        "spots_meet_count": len(spots_meet),
        "spots_under_count": len(spots_under),
        "spots_under_runs": spots_under,
        "avg_meet_count": len(avg_meet),
        "avg_under_count": len(avg_under),
        "avg_under_runs": avg_under,
    }


counts_records = []
failed_spots_records = []
failed_length_records = []

for acc in PROJECTS:
    result = validate_project(DATA_DIR / f"{acc}_runinfo.csv")

    counts_records.append({
        "BioProject": acc,
        "Total Runs": result["total_runs"],
        f"Spots Passed (>= {MIN_SPOTS:,})": result["spots_meet_count"],
        f"Spots Failed (< {MIN_SPOTS:,})": result["spots_under_count"],
        f"Length Passed (>= {MIN_AVG_LENGTH})": result["avg_meet_count"],
        f"Length Failed (< {MIN_AVG_LENGTH})": result["avg_under_count"],
        f"Runs >= {MIN_RUNS}": "PASS" if result["passes_min_runs"] else "FAIL",
        "Paired/Total": f"{result['paired_runs']}/{result['total_runs']}",
    })

    failed_spots_records.append({
        "BioProject": acc,
        f"Failed Runs by Spots (< {MIN_SPOTS:,})": ", ".join(result["spots_under_runs"]) if result["spots_under_runs"] else "-",
    })

    failed_length_records.append({
        "BioProject": acc,
        f"Failed Runs by avgLength (< {MIN_AVG_LENGTH})": ", ".join(result["avg_under_runs"]) if result["avg_under_runs"] else "-",
    })

validation_counts = pd.DataFrame(counts_records).sort_values("BioProject").reset_index(drop=True)
validation_failed_spots = pd.DataFrame(failed_spots_records).sort_values("BioProject").reset_index(drop=True)
validation_failed_length = pd.DataFrame(failed_length_records).sort_values("BioProject").reset_index(drop=True)

print("Validation Summary")
try:
    display(validation_counts.style.hide(axis="index").set_properties(**{"text-align": "left"}))
except Exception:
    display(validation_counts)

print("Failed Runs by Spots")
try:
    display(validation_failed_spots.style.hide(axis="index").set_properties(**{"text-align": "left"}))
except Exception:
    display(validation_failed_spots)

print("Failed Runs by avgLength")
try:
    display(validation_failed_length.style.hide(axis="index").set_properties(**{"text-align": "left"}))
except Exception:
    display(validation_failed_length)


Validation Summary


BioProject,Total Runs,"Spots Passed (>= 40,000,000)","Spots Failed (< 40,000,000)",Length Passed (>= 300),Length Failed (< 300),Runs >= 20,Paired/Total
PRJNA1017789,26,26,0,26,0,PASS,26/26
PRJNA1277581,30,30,0,30,0,PASS,30/30
PRJNA717662,152,152,0,89,63,PASS,152/152


Failed Runs by Spots


BioProject,"Failed Runs by Spots (< 40,000,000)"
PRJNA1017789,-
PRJNA1277581,-
PRJNA717662,-


Failed Runs by avgLength


BioProject,Failed Runs by avgLength (< 300)
PRJNA1017789,-
PRJNA1277581,-
PRJNA717662,"SRR15510544, SRR15510545, SRR15510546, SRR15510547, SRR15510548, SRR15510549, SRR15510550, SRR15510551, SRR15510552, SRR15510553, SRR15510554, SRR15510555, SRR15510556, SRR15510557, SRR15510558, SRR15510559, SRR15510560, SRR15510561, SRR15510562, SRR15510563, SRR15510564, SRR15510565, SRR15510566, SRR15510567, SRR15510568, SRR15510569, SRR15510570, SRR15510571, SRR15510572, SRR15510573, SRR15510574, SRR15510575, SRR15510576, SRR15510577, SRR15510578, SRR15510579, SRR15510580, SRR15510581, SRR15510582, SRR15510583, SRR15510584, SRR15510585, SRR15510586, SRR15510587, SRR15510588, SRR15510589, SRR15510590, SRR15510591, SRR15510592, SRR15510593, SRR15510594, SRR15510595, SRR15510596, SRR15510597, SRR15510598, SRR15510599, SRR15510600, SRR15510601, SRR15510602, SRR15510603, SRR15510604, SRR15510542, SRR15510543"


## Section 4 - Summary and Dataset Choice

### What each dataset is
- **PRJNA717662 (Human blood, post-COVID time course):** Large longitudinal transcriptome study with healthy controls and recovered patients at 12, 16, and 24 weeks.
- **PRJNA1017789 (Mouse DRG injury model):** Controlled perturbation study focused on AhR signaling and axon regeneration after nerve injury.
- **PRJNA1277581 (Zebrafish retina regeneration):** Regeneration-focused perturbation study on Muller glia-microglia signaling across injury/treatment conditions.

### What we observed from validation
- All three projects pass the **minimum sample requirement** (`>=20 runs`) and show strong sequencing depth (`spots >= 40M` for all runs in current tables).
- **Read-length caveat:** `PRJNA717662` includes runs with `avgLength < 300` (63 runs), while `PRJNA1017789` and `PRJNA1277581` do not show this issue.
- All three are paired-end transcriptomic datasets and are suitable for downstream RNA-seq workflows.

### Better choice based on findings and purpose
- **Best overall choice for translational human-focused analysis:** `PRJNA717662`
  - Why: largest cohort, human samples, time-course design, strong relevance for clinically meaningful differential expression.
- **Best choice for cleaner technical consistency (read length):** `PRJNA1017789` or `PRJNA1277581`
  - Why: no current `avgLength < 300` runs in the RunInfo tables.

### Working recommendation
- For **human disease biology and stronger statistical power**, prioritize `PRJNA717662` and filter to qualifying runs as needed.
- Keep `PRJNA1017789` and `PRJNA1277581` as strong secondary/contrast projects for mechanism-focused analyses.


## Next Analysis Steps
1. Freeze and save the filtered SRR lists per project for reproducibility.
2. Proceed to FASTQ download + QC only after validation criteria are finalized.
3. Start alignment/quantification workflow on validated SRR subsets.
